In [1]:
import glob
import nbformat
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import hashlib
import sys
import platform
from bs4 import BeautifulSoup
import re
import copy
import os

In [2]:
fpaths = glob.glob('../../accy575-sp-2023/05-yellow-cab-submissions/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['../../accy575-sp-2023/05-yellow-cab-submissions\\adediranboye_LATE_324892_8254893_05-yellow-cab-case-2.ipynb',
 '../../accy575-sp-2023/05-yellow-cab-submissions\\aisaaidana_334459_8254867_aaisa2_yellow_cab.ipynb',
 '../../accy575-sp-2023/05-yellow-cab-submissions\\amirhafsa_7733_8253203_hamir2_yellow_cab.ipynb']

In [3]:
fpaths = glob.glob('test-notebooks/exercise-05/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['test-notebooks/exercise-05\\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb',
 'test-notebooks/exercise-05\\esparzajoel_51499_7451315_Copy_of_exercise_05_pandas_filtering_sorting.ipynb',
 'test-notebooks/exercise-05\\tabornatalie_8175_7404582_exercise_05_pandas_filtering_sorting.ipynb']

In [4]:
fpaths = glob.glob('test-notebooks/05-yellow*-SOLUTION.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['test-notebooks\\05-yellow-cab-case-SOLUTION.ipynb']

In [5]:
fpaths = glob.glob('../submissions/exercise-05/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:3]

['../submissions/exercise-05\\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb',
 '../submissions/exercise-05\\esparzajoel_51499_7451315_Copy_of_exercise_05_pandas_filtering_sorting.ipynb',
 '../submissions/exercise-05\\tabornatalie_8175_7404582_exercise_05_pandas_filtering_sorting.ipynb']

In [6]:
fpaths = fpaths[:1]
fpaths

['../submissions/exercise-05\\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb']

In [7]:
import lambdagrader

graded_results = []

for notebook_path in fpaths:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        test_cases_hash = lambdagrader.get_test_cases_hash(nb)
        
        lambdagrader.preprocess_test_case_cells(nb)
        lambdagrader.add_grader_scripts(nb)

        p = Path(notebook_path)
        filestem = p.name
        
        print(f'Grading {notebook_path}')
        
        ep = ExecutePreprocessor(
            timeout=1800,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        
        # save graded notebook
        converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        
        # running the notebook will store the graded result to a JSON file
        # rename graded result JSON file
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        
        # read graded result to generate a summary
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
        
        # add filename
        # we add it here instead of trying to add it within the Jupyter notebook
        # because it is tricky to grab the current file name inside a Jupyter kernel
        graded_result['filename'] = Path(notebook_path).name
        
        # MD5 hash of the submitted Jupyter notebook file
        # this can be used to detect duplicate submission to prevent unnecessary re-grading
        with open(notebook_path, 'rb') as f:
            graded_result['submission_notebook_hash'] = hashlib.md5(f.read()).hexdigest()
        
        # MD5 hash of test cases code
        # this helps us to identify any potential cases
        # where a learner has modified or deleted the test cases code cell
        graded_result['test_cases_hash'] = test_cases_hash
        
        # store Python version and platform used to run the notebook
        graded_result['grader_python_version'] = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
        graded_result['grader_platform'] = platform.platform()

        # save updated JSON to file
        with open(graded_result_json_path, 'w') as f:
            json.dump(graded_result, f, indent=2)
            
        # clean up notebook
        lambdagrader.remove_grader_scripts(nb)
        lambdagrader.add_graded_result(nb, graded_result)
           
        # extract user code to a Python file
        extracted_user_code = lambdagrader.extract_user_code_from_notebook(nb)
        extracted_code_path = notebook_path.replace('.ipynb', '_user_code.py')
        
        with open(extracted_code_path, "w", encoding="utf-8") as f:
            f.write(extracted_user_code)
        
        # store graded result to HTML
        filestem = Path(notebook_path).name
        graded_html_path = notebook_path.replace('.ipynb', '-graded.html')
        lambdagrader.save_graded_notebook_to_html(
            nb,
            html_title=filestem,
            output_path=graded_html_path,
            graded_result=graded_result
        )
        
        # LOCAL ENVIRONMENT ONLY
        # the Lambda handler only processes one file instead of
        # running a batch
        # the code below generates a CSV to show results for multiple files
        # get text summary of user's graded result
        text_summary = lambdagrader.generate_text_summary(graded_result)
        
        result_summary = graded_result.copy()
        result_summary['text_summary'] = text_summary
        del result_summary['results']
        graded_results.append(result_summary)
        
        print(f'Complete')
    except Exception as e:
        print(f'Error while grading {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)

df_summary.to_csv(
    f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
    index=None
)

Grading ../submissions/exercise-05\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb
Complete


In [8]:
from copydetect import CopyDetector
detector = CopyDetector(test_dirs=["../submissions/exercise-07"], extensions=["py"])
# detector.run()
# detector.generate_html_report()

In [9]:
# from lambdagrader import *

notebook_path = '../notebooks\\case-study-04-rideshare-trips-SOLUTION.ipynb'
nb = nbformat.read(notebook_path, as_version=4)

tcs = lambdagrader.extract_test_cases_metadata_from_notebook(nb)

s = 0

for o in tcs:
    s += o['points']
    
print(s)

80.0
